# TODO:
#### João:
- [x] cena das datas passar horas para tarde/noite/manha/madrugada && dividr dias do mes e mes // Semi Done not sure what to do with month and days divider
- [x] nominal to numeric | NOminal value discretization using label Encoding
- [] outliers
- [] noise  NOTE: Binning is a technique for data smoothing. Data smoothing is employed to removenoise from data.
- [x] duplicated Records 

#### Jorge
- [x] Binning
- [x] feature scalling
- [x] svn
- [x] training and testing data

# Dados e Aprendizagem Automática
## Trabalho Prático
### Grupo 13
### Membros:
- (PG50304) Cristiano Pereira
- (PG50463) João Martins
- (PG50506) Jorge Lima
- (PG50733) Rúben Santos

Notebook structure:
- 1. Importing libraries
- 2. Loading the dataset
- 3. Exploratory Data Analysis TODO Joao
- 4. Data Preprocessing TODO Joao
- 5. Model Training TODO Jorge
- 6. Model Evaluation TODO Jorge
- 8. Conclusion Todo

## 1. Importing libraries

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing, tree, metrics, model_selection, linear_model
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


# Lois
from lois import lois_ds_report
%matplotlib inline

## 2. Loading the dataset

In [ ]:
trainingData = pd.read_csv('data/training_data.csv')
testData = pd.read_csv('data/test_data.csv')

## 3. Exploratory Data Analysis TODO Joao

In [ ]:
lois_ds_report(trainingData,target_variable="incidents")

In [ ]:
trainingData.head()

In [ ]:
trainingData.describe()

In [ ]:
trainingData.info()

## 4. Data Preprocessing TODO Joao

### Look for any Duplicated Values

In [ ]:

trainingData[trainingData.duplicated()]


### Handle missing values

In [ ]:
# collum affected_roles to numeric

def count_roads(affected_roads):
    #check if affected_roads is a float(nan values)
    if isinstance(affected_roads, float) and np.isnan(affected_roads):
        return 0
    return len(affected_roads.split(','))

# count the number of different roads affected
def count_roads(affected_roads):
    #check if affected_roads is a float(nan values)
    if isinstance(affected_roads, float) and np.isnan(affected_roads):
        return 0
    result = []
    return len([result.append(x) for x in affected_roads.split(',') if x not in result] )


trainingData['number_affected_roads'] = trainingData['affected_roads'].apply(count_roads)
testData['number_affected_roads'] = testData['affected_roads'].apply(count_roads)
trainingData['number_different_affected_roads'] = trainingData['affected_roads'].apply(count_roads)
testData['number_different_affected_roads'] = testData['affected_roads'].apply(count_roads)

print(trainingData.head())

---

In [ ]:
#testing if exists null values
# TODO: Coorect to drop the affected_roads collumn?
trainingData.drop('affected_roads', axis=1, inplace=True)
testData.drop('affected_roads', axis=1, inplace=True)
trainingData.isna().any()

In [ ]:
sns.pairplot(trainingData)

In [ ]:
trainingData.profile_report(minimal=True)

##### Dropping city_name because it's only data from Guimaraes

In [ ]:
#Remove city_name column, only guimaraes is present
trainingData.drop('city_name',axis=1,inplace=True)
testData.drop('city_name',axis=1,inplace=True)


# Turn values to Numeric

###  Turn nominal values to numeric

In [ ]:
dict_magnitude_of_delay = {'UNDEFINED': 0, 'MODERATE':1, 'MAJOR':3}
trainingData['magnitude_of_delay'] = trainingData['magnitude_of_delay'].map(dict_magnitude_of_delay)
testData['magnitude_of_delay'] = testData['magnitude_of_delay'].map(dict_magnitude_of_delay)

dict_luminosity = {'LOW_LIGHT': 0, 'LIGHT':1, 'DARK':2}
trainingData['luminosity'] = trainingData['luminosity'].map(dict_luminosity)
testData['luminosity'] = testData['luminosity'].map(dict_luminosity)

dict_avg_rain = {'Sem Chuva': 0, 'chuva fraca': 1, 'chuva moderada': 2, 'chuva forte': 3}
trainingData['avg_rain'] = trainingData['avg_rain'].map(dict_avg_rain)
testData['avg_rain'] = testData['avg_rain'].map(dict_avg_rain)

#### categorizing time of day

In [ ]:
# Turn column record_date into manha,tarde,noite or madrugada
def turn_date_into_time_of_day(record_date):
    hour = int(record_date.split(' ')[1].split(':')[0])
    if hour >= 6 and hour < 12:
        return 'manha'
    elif hour >= 12 and hour < 18:
        return 'tarde'
    elif hour >= 18 and hour < 24:
        return 'noite'
    else:
        return 'madrugada'


trainingData['time_of_day'] = trainingData['record_date'].apply(turn_date_into_time_of_day)
testData['time_of_day'] = testData['record_date'].apply(turn_date_into_time_of_day)

dict_time_of_day = {'manha': 0, 'tarde':1, 'noite':2, 'madrugada':3}
trainingData['time_of_day'] = trainingData['time_of_day'].map(dict_time_of_day)
testData['time_of_day'] = testData['time_of_day'].map(dict_time_of_day)

In [ ]:
#add 2 column with the month and day from time_of_day
trainingData['month'] = pd.DatetimeIndex(trainingData['record_date']).month
testData['month'] = pd.DatetimeIndex(testData['record_date']).month
trainingData['day'] = pd.DatetimeIndex(trainingData['record_date']).day
testData['day'] = pd.DatetimeIndex(testData['record_date']).day

In [ ]:
corr_matrix = trainingData.corr(numeric_only=True)
sns.heatmap(corr_matrix,vmin=1,vmax=1,square=True, annot=True)

### Binning

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

def apply_binning(data, column, n_bins, encode='ordinal', strategy='quantile'):
    if column in data.columns:
        estimater = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy)
        data[column + "_binned"] = estimater.fit_transform(data[[column]])
        data.drop(column, axis=1, inplace=True)


#apply_binning(testingData, 'avg_atm_pressure', 5)

### Normalization

In [ ]:
# Normalization
from sklearn.preprocessing import MinMaxScaler

def apply_normalization(data, column):
    if column in data.columns:
        scaler = MinMaxScaler()
        data[column + "_normalized"] = scaler.fit_transform(data[[column]])
        data.drop(column, axis=1, inplace=True)

#apply_normalization(testingData, 'delay_in_seconds')

### Standardization

In [ ]:
# Standardization
from sklearn.preprocessing import StandardScaler

def apply_standardization(data, column):
    if column in data.columns:
        scaler = StandardScaler()
        data[column + "_standardized"] = scaler.fit_transform(data[[column]])
        data.drop(column, axis=1, inplace=True)

#apply_standardization(testingData, 'avg_humidity')

## 5. Models Training TODO Jorge

In [ ]:
""" Save results to a file. """
def saveResults ( results, path ):
    file = open(path, 'w')
    file.write ("RowId,Incidents\n")
    i = 1
    for result in results :
        file.write (str(i) + "," + result + "\n")
        i += 1

In [ ]:
# run for test or run for submission
results_to_test = True

# split data into train and test
trainData = trainingData.copy()
ttData = testData.copy()

if results_to_test:
    y = trainData['incidents']
    x = trainData.drop(['incidents'], axis=1)
    x = x.drop(['record_date'], axis=1) # TODO REMOVER
    x_training, x_testing, y_training, y_testing = model_selection.train_test_split(x, y, test_size=0.5, random_state=2022)

else :
    x_testing = ttData.drop(['record_date'], axis=1) # TODO REMOVER
    y_training = trainData['incidents'] 
    x_training = trainData.drop(['incidents'], axis=1)
    x_training = x_training.drop(['record_date'], axis=1) # TODO REMOVER

### 5.1 Regressão Logística

In [ ]:
logmodel = linear_model.LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
logmodel.fit(x_training,y_training)
logistic_regression_predictions = logmodel.predict(x_testing)

### 5.2 Árvores de Decisão e Classificação

In [ ]:
dtc = tree.DecisionTreeClassifier(random_state=2022, criterion="gini", max_depth=10)
dtc.fit(x_training,y_training)
decision_trees_predictions = dtc.predict(x_testing)

### 5.3 Maquinas de Vectores de Suporte (SVM)

In [ ]:
df_feat = pd.DataFrame(x_training, columns=x_training.columns)
df_target = pd.DataFrame(y_training, columns=['incidents'])

# 10-Fold Cross Validation
cross_valid_model = SVC(random_state=2022)
cross_val_score(cross_valid_model, df_feat, np.ravel(df_target), cv=10)

### 5.4 Hold-out

In [ ]:
model = SVC(random_state=2022)
model.fit(x_training, y_training)
hold_out_predictions = model.predict(x_testing)

### 5.5 Grid Search

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['rbf']}
grid = GridSearchCV(SVC(random_state=2022), param_grid, refit=True, verbose=3)
grid.fit(x_training, y_training)
grid_predictions = grid.predict(x_testing)

### 5.6 KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_training, y_training)
knn_predictions = knn.predict(x_testing)

### 5.7 GaussianProcessClassifier

In [ ]:
gaussian = GaussianProcessClassifier(n_restarts_optimizer=0, max_iter_predict=100, random_state=2022)
gaussian.fit(x_training, y_training)
gaussian_predictions = gaussian.predict(x_testing)

### 5.8 RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, random_state=2022)
random_forest.fit(x_training, y_training)
random_forest_predictions = random_forest.predict(x_testing)

### 5.9 MLPClassifier

In [ ]:
mlpclass = MLPClassifier(alpha=1, max_iter=1000)
mlpclass.fit(x_training, y_training)
mlpclass_predictions = mlpclass.predict(x_testing)

### 5.10 AdaBoostClassifier

In [ ]:
adaBoost = AdaBoostClassifier(n_estimators=100, random_state=2022)
adaBoost.fit(x_training, y_training)
adaBoost_predictions = adaBoost.predict(x_testing)

### 5.11 GaussianNB

In [ ]:
gaussianNB = GaussianNB()
gaussianNB.fit(x_training, y_training)
gaussianNB_predictions = gaussianNB.predict(x_testing)

### 5.12 QuadraticDiscriminantAnalysis

In [ ]:
quadraticDiscriminantAnalysis = QuadraticDiscriminantAnalysis()
quadraticDiscriminantAnalysis.fit(x_training, y_training)
quadraticDiscriminantAnalysis_predictions = quadraticDiscriminantAnalysis.predict(x_testing)

## 6. Model Evaluation TODO Jorge

In [ ]:
if results_to_test:

    results = []
    results_table = []

    # 5.1 Regressão Logística
    logistic_regression_classification = metrics.classification_report(y_testing, logistic_regression_predictions, zero_division=0)
    logistic_regression_classification_dict = metrics.classification_report(y_testing, logistic_regression_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Regressão Logística', 'table': logistic_regression_classification})
    results.append({
        'model': 'Regressão Logística',
        'accuracy': logistic_regression_classification_dict['accuracy'],
        'precision': logistic_regression_classification_dict['macro avg']['precision'],
        'recall': logistic_regression_classification_dict['macro avg']['recall'],
        'f1-score': logistic_regression_classification_dict['macro avg']['f1-score'],
        })

    # 5.2 Decision Tree
    decision_trees_classification = metrics.classification_report(y_testing, decision_trees_predictions, zero_division=0)
    decision_trees_classification_dict = metrics.classification_report(y_testing, decision_trees_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Decision Tree', 'table': decision_trees_classification})
    results.append({
        'model': 'Decision Tree',
        'accuracy': decision_trees_classification_dict['accuracy'],
        'precision': decision_trees_classification_dict['macro avg']['precision'],
        'recall': decision_trees_classification_dict['macro avg']['recall'],
        'f1-score': decision_trees_classification_dict['macro avg']['f1-score']
        })

    # 5.4 Hold Out
    hold_out_classification = metrics.classification_report(y_testing, hold_out_predictions, zero_division=0)
    hold_out_classification_dict = metrics.classification_report(y_testing, hold_out_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Hold Out', 'table': hold_out_classification})
    results.append({
        'model': 'Hold Out',
        'accuracy': hold_out_classification_dict['accuracy'],
        'precision': hold_out_classification_dict['macro avg']['precision'],
        'recall': hold_out_classification_dict['macro avg']['recall'],
        'f1-score': hold_out_classification_dict['macro avg']['f1-score']
        })

    # 5.5 Grid Search
    grid_classification = metrics.classification_report(y_testing, grid_predictions, zero_division=0)
    grid_classification_dict = metrics.classification_report(y_testing, grid_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Grid Search', 'table': grid_classification})
    results.append({
        'model': 'Grid Search',
        'accuracy': grid_classification_dict['accuracy'],
        'precision': grid_classification_dict['macro avg']['precision'],
        'recall': grid_classification_dict['macro avg']['recall'],
        'f1-score': grid_classification_dict['macro avg']['f1-score']
        })

    #  5.6 KNN
    knn_classification = metrics.classification_report(y_testing, knn_predictions, zero_division=0)
    knn_classification_dict = metrics.classification_report(y_testing, knn_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'KNN', 'table': knn_classification})
    results.append({
        'model': 'KNN',
        'accuracy': knn_classification_dict['accuracy'],
        'precision': knn_classification_dict['macro avg']['precision'],
        'recall': knn_classification_dict['macro avg']['recall'],
        'f1-score': knn_classification_dict['macro avg']['f1-score']
        })

    # 5.7 Gaussian
    gaussian_classification = metrics.classification_report(y_testing, gaussian_predictions, zero_division=0)
    gaussian_classification_dict = metrics.classification_report(y_testing, gaussian_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Gaussian', 'table': gaussian_classification})
    results.append({
        'model': 'Gaussian',
        'accuracy': gaussian_classification_dict['accuracy'],
        'precision': gaussian_classification_dict['macro avg']['precision'],
        'recall': gaussian_classification_dict['macro avg']['recall'],
        'f1-score': gaussian_classification_dict['macro avg']['f1-score']
        })

    # 5.8 Random Forest
    random_forest_classification = metrics.classification_report(y_testing, random_forest_predictions, zero_division=0)
    random_forest_classification_dict = metrics.classification_report(y_testing, random_forest_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Random Forest', 'table': random_forest_classification})
    results.append({
        'model': 'Random Forest',
        'accuracy': random_forest_classification_dict['accuracy'],
        'precision': random_forest_classification_dict['macro avg']['precision'],
        'recall': random_forest_classification_dict['macro avg']['recall'],
        'f1-score': random_forest_classification_dict['macro avg']['f1-score']
        })

    # 5.9 MLP
    mlp_classification = metrics.classification_report(y_testing, mlpclass_predictions, zero_division=0)
    mlp_classification_dict = metrics.classification_report(y_testing, mlpclass_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'MLP', 'table': mlp_classification})
    results.append({
        'model': 'MLP',
        'accuracy': mlp_classification_dict['accuracy'],
        'precision': mlp_classification_dict['macro avg']['precision'],
        'recall': mlp_classification_dict['macro avg']['recall'],
        'f1-score': mlp_classification_dict['macro avg']['f1-score']
        })
    
    # 5.10 AdaBoost
    adaBoost_classification = metrics.classification_report(y_testing, adaBoost_predictions, zero_division=0)
    adaBoost_classification_dict = metrics.classification_report(y_testing, adaBoost_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'AdaBoost', 'table': adaBoost_classification})
    results.append({
        'model': 'AdaBoost',
        'accuracy': adaBoost_classification_dict['accuracy'],
        'precision': adaBoost_classification_dict['macro avg']['precision'],
        'recall': adaBoost_classification_dict['macro avg']['recall'],
        'f1-score': adaBoost_classification_dict['macro avg']['f1-score']
        })

    # 5.11 GaussianNB
    gaussianNB_classification = metrics.classification_report(y_testing, gaussianNB_predictions, zero_division=0)
    gaussianNB_classification_dict = metrics.classification_report(y_testing, gaussianNB_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'GaussianNB', 'table': gaussianNB_classification})
    results.append({
        'model': 'GaussianNB',
        'accuracy': gaussianNB_classification_dict['accuracy'],
        'precision': gaussianNB_classification_dict['macro avg']['precision'],
        'recall': gaussianNB_classification_dict['macro avg']['recall'],
        'f1-score': gaussianNB_classification_dict['macro avg']['f1-score']
        })
    
    # 5.12 Quadratic Discriminant Analysis
    qda_classification = metrics.classification_report(y_testing, quadraticDiscriminantAnalysis_predictions, zero_division=0)
    qda_classification_dict = metrics.classification_report(y_testing, quadraticDiscriminantAnalysis_predictions, output_dict=True, zero_division=0)
    results_table.append({'model': 'Quadratic Discriminant Analysis', 'table': qda_classification})
    results.append({
        'model': 'Quadratic Discriminant Analysis',
        'accuracy': qda_classification_dict['accuracy'],
        'precision': qda_classification_dict['macro avg']['precision'],
        'recall': qda_classification_dict['macro avg']['recall'],
        'f1-score': qda_classification_dict['macro avg']['f1-score']
        })

else:
    saveResults(logistic_regression_predictions, 'results/logisticRegression.csv')
    saveResults(decision_trees_predictions, 'results/classificationDecisionTree.csv')
    saveResults(hold_out_predictions, 'results/Hold-Out-SVN.csv')
    saveResults(grid_predictions, 'results/GridSearchCV.csv')
    saveResults(knn_predictions, 'results/KNN.csv')
    saveResults(gaussian_predictions, 'results/Gaussian.csv')
    saveResults(random_forest_predictions, 'results/RandomForest.csv')
    saveResults(mlpclass_predictions, 'results/MLP.csv')
    saveResults(adaBoost_predictions, 'results/AdaBoost.csv')
    saveResults(gaussianNB_predictions, 'results/GaussianNB.csv')
    saveResults(quadraticDiscriminantAnalysis_predictions, 'results/QuadraticDiscriminantAnalysis.csv')


In [ ]:
for result in results_table:
    print("----------" + result['model'] + "----------------------------------------------------------------")
    print(result['table'])

In [ ]:
results_plot = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1-Score'])
for row in results:
    results_plot = results_plot.append({
        'Model': row['model'],
        'Accuracy': row['accuracy'],
        'Precision': row['precision'],
        'Recall': row['recall'],
        'F1-Score': row['f1-score']
        }, ignore_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5), dpi=100)
sns.barplot(x=results_plot['Model'] , y=results_plot['Accuracy'], ax=ax, palette='Set2')
ax.set_xlabel('Model', fontdict={'color':'black', 'weight':'bold', 'size': 16})
ax.set_ylabel('Accuracy', fontdict={'color':'black', 'weight':'bold', 'size': 16})

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5), dpi=100)
sns.barplot(x=results_plot['Model'] , y=results_plot['Precision'], ax=ax, palette='Set2')
ax.set_xlabel('Model', fontdict={'color':'black', 'weight':'bold', 'size': 16})
ax.set_ylabel('Precision', fontdict={'color':'black', 'weight':'bold', 'size': 16})

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5), dpi=100)
sns.barplot(x=results_plot['Model'] , y=results_plot['Recall'], ax=ax, palette='Set2')
ax.set_xlabel('Model', fontdict={'color':'black', 'weight':'bold', 'size': 16})
ax.set_ylabel('Recall', fontdict={'color':'black', 'weight':'bold', 'size': 16})

In [ ]:
fig, ax = plt.subplots(figsize=(25, 5), dpi=100)
sns.barplot(x=results_plot['Model'] , y=results_plot['F1-Score'], ax=ax, palette='Set2')
ax.set_xlabel('Model', fontdict={'color':'black', 'weight':'bold', 'size': 16})
ax.set_ylabel('F1-Score', fontdict={'color':'black', 'weight':'bold', 'size': 16})